In [ ]:
import os
import glob
import numpy as np
import math
from scipy.optimize import linear_sum_assignment

# Calculation of location similarity LocSim
def loc_sim(P_pos, G_pos, tau):
    """
    Function to compute location similarity LocSim
    
    \[
    \operatorname{LocSim}(P, G) = \exp\!\left( \ln(0.05) \cdot \frac{\|P - G\|_2^2}{\tau^2} \right)
    \]
    
    :param P_pos: Predicted position (coordinates in list or numpy array)
    :param G_pos: Ground Truth Location
    :param tau: normalized parameter
    :return: LocSim Value
    """
    P_pos = np.array(P_pos)
    G_pos = np.array(G_pos)
    distance_sq = np.sum((P_pos - G_pos)**2)
    sim = math.exp(math.log(0.05) * (distance_sq / (tau**2)))
    return sim

# Calculation of attribute similarity IdSim
def id_sim(P_attr, G_attr):
    """
    Function to calculate attribute similarity IdSim
    \[
    \operatorname{IdSim}(P, G) =
    \begin{cases}
    1, & \text{if } P_{\text{attr}} = G_{\text{attr}}, \\
    0, & \text{otherwise.}
    \end{cases}
    \]
    :param P_attr: Attributes of prediction (specify each attribute in a dictionary, etc.)
    :param G_attr: Ground Truth Attributes
    :return: 1 if all attributes match, 0 otherwise
    """

    return 1 if P_attr == G_attr else 0

# Overall similarity of each pair
def sim_ti_hota(P, G, tau):
    """
    Function to calculate the overall similarity of TI-HOTA
    \[
    \operatorname{Sim}_{\mathrm{TI\text{-}HOTA}}(P, G) = \operatorname{LocSim}(P, G) \times \operatorname{IdSim}(P, G)
    \]
    :param P: Prediction Information {'position': [x, y, ...], 'attributes': {...}} 
    :param G: Ground Truth Information {'position': [x, y, ...], 'attributes': {...}} 
    :param tau: normalized parameter for loc_sim calculation
    :return: overall similarity Sim_{TI-HOTA}
    """
    
    return loc_sim(P['position'], G['position'], tau) * id_sim(P['attributes'], G['attributes'])

def compute_ti_hota_alpha(frames, tau, alpha):
    total_tp = 0      # True Positives 
    total_fp = 0      # False Positives
    total_fn = 0      # False Negatives

    # Prepare a dictionary for counting track-by-track associations here
    # assoc_counts: key = GT attributes, value = {'TPA': count, 'FNA': count}
    assoc_counts = {}
    # fpa_counts: key = attributes (from prediction), value = count of unmatched predictions for that attributes
    fpa_counts = {}

    for frame in frames:
        gt_objects = frame.get('ground_truths', [])
        pred_objects = frame.get('predictions', [])
        n_gt = len(gt_objects)
        n_pred = len(pred_objects)
        
        # Skip if both are empty in the frame
        if n_gt == 0 and n_pred == 0:
            continue
        if n_gt == 0:
            total_fp += n_pred
            # All prediction objects are FP → fpa_counts updated
            for pred in pred_objects:
                pid = pred['attributes']
                fpa_counts[pid] = fpa_counts.get(pid, 0) + 1
            continue
        if n_pred == 0:
            total_fn += n_gt
            # All GT objects are missed → association is marked as failed
            for gt in gt_objects:
                gid = gt['attributes']
                if gid not in assoc_counts:
                    assoc_counts[gid] = {'TPA': 0, 'FNA': 0}
                assoc_counts[gid]['FNA'] += 1
            continue
        
        # Matching in each frame (overall score considering position and attributes)
        cost_matrix = np.zeros((n_gt, n_pred))
        combined_sim_matrix = np.zeros((n_gt, n_pred))
        
        for i, gt in enumerate(gt_objects):
            for j, pred in enumerate(pred_objects):
                # Here the cost is based on location similarity, and the attributes are used later for track association
                loc = loc_sim(pred['position'], gt['position'], tau)
                # Overall score: loc for attribute match, 0 otherwise
                id_val = 1 if gt['attributes'] == pred['attributes'] else 0
                combined = loc * id_val
                combined_sim_matrix[i, j] = combined
                cost_matrix[i, j] = 1 - combined  # For cost minimization
                
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        
        matched_indices = []
        # Of the matching results, if the overall score is above the threshold, it is considered a valid match.
        for i, j in zip(row_ind, col_ind):
            if combined_sim_matrix[i, j] >= alpha:
                matched_indices.append((i, j))
        
        tp = len(matched_indices)
        fp = n_pred - tp
        fn = n_gt - tp
        
        total_tp += tp
        total_fp += fp
        total_fn += fn
        
        # First, for each GT object, record whether a match was made
        # If no match, add 1 to FNA
        matched_gt_indices = {i for i, j in matched_indices}
        for i, gt in enumerate(gt_objects):
            gid = gt['attributes']
            if gid not in assoc_counts:
                assoc_counts[gid] = {'TPA': 0, 'FNA': 0}
            if i not in matched_gt_indices:
                assoc_counts[gid]['FNA'] += 1 # False Negative Association
        # Then, for each match pair, update TPA/FNA by whether the ID of the prediction matches the GT
        for i, j in matched_indices:
            gt = gt_objects[i]
            pred = pred_objects[j]
            gid = gt['attributes']
            # It should have already been initialized.
            if gt['attributes'] not in assoc_counts:
                assoc_counts[gid] = {'TPA': 0, 'FNA': 0}
            if gt['attributes'] == pred['attributes']:
                assoc_counts[gid]['TPA'] += 1
            else:
                assoc_counts[gid]['FNA'] += 1

        # In addition, unmatched predicted objects become FPs, which are recorded as FPAs
        matched_pred_indices = {j for i, j in matched_indices}
        for j, pred in enumerate(pred_objects):
            pid = pred['attributes']
            if j not in matched_pred_indices:
                fpa_counts[pid] = fpa_counts.get(pid, 0) + 1

    # Detection accuracy (DetA)_α = TP / (TP + FP + FN)
    total_detections = total_tp + total_fp + total_fn
    det_acc = total_tp / total_detections if total_detections > 0 else 0

    # Association accuracy: for each GT track c A(c) = TPA(c) / (TPA(c) + FNA(c) + FPA(c))
    sum_A = 0
    n_tracks = 0
    for gid, counts in assoc_counts.items():
        TPA = counts.get('TPA', 0)
        FNA = counts.get('FNA', 0)
        FPA = fpa_counts.get(gid, 0)
        denom = TPA + FNA + FPA
        if denom > 0:
            A_c = TPA / denom
            sum_A += A_c
            n_tracks += 1
    ass_acc = sum_A / n_tracks if n_tracks > 0 else 0

    # TI-HOTA_α = sqrt(DetA_α * AssA_α)
    ti_hota = math.sqrt(det_acc * ass_acc)

    return ti_hota, det_acc, ass_acc, total_tp, total_fp, total_fn, assoc_counts, fpa_counts

# The function that calculates TI-HOTA for each threshold value α and takes the mean of the TI-HOTA_α
def compute_ti_hota(frames, tau):
    alphas = np.arange(0.05, 1.0, 0.05)
    ti_hota_alphas = []
    det_acc_values = []
    ass_acc_values = []
    total_tp_values = []
    total_fp_values = []
    total_fn_values = []
    for alpha in alphas:
        ti_hota, det_acc, ass_acc, tp, fp, fn, _, _ = compute_ti_hota_alpha(frames, tau, alpha)
        ti_hota_alphas.append(ti_hota)
        det_acc_values.append(det_acc)
        ass_acc_values.append(ass_acc)
        total_tp_values.append(tp)
        total_fp_values.append(fp)
        total_fn_values.append(fn)
    ti_hota = np.mean(ti_hota_alphas)
    mean_det_acc = np.mean(det_acc_values)
    mean_ass_acc = np.mean(ass_acc_values)
    mean_total_tp = np.mean(total_tp_values)
    mean_total_fp = np.mean(total_fp_values)
    mean_total_fn = np.mean(total_fn_values)
    
    metrics = {
        "TI-HOTA": ti_hota,
        "TI-HOTA_alphas": ti_hota_alphas,
        "DetA": mean_det_acc,
        "AssA": mean_ass_acc,
        "TP": mean_total_tp,
        "FP": mean_total_fp,
        "FN": mean_total_fn,
    }
    return metrics, alphas

In [ ]:
# Function to read txt files of round truth and prediction results and group them by frames
def load_frames(file_path):
    frames_dict = {}
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split(',')
            if len(parts) < 5:
                continue
            # Row contents: frame, ID, x, y, attribute
            frame = int(parts[0])
            id = int(parts[1])
            x = float(parts[2])
            y = float(parts[3])
            attributes = parts[4]
            obj = {'position': [x, y], 'id': id, 'attributes': attributes}
            if frame not in frames_dict:
                frames_dict[frame] = []
            frames_dict[frame].append(obj)
    return frames_dict

# Reads the ground truth file of the same name and the prediction result file and merges them frame by frame
def load_video(gt_file, pred_file):
    gt_frames = load_frames(gt_file)
    pred_frames = load_frames(pred_file)
    all_frame_ids = sorted(set(gt_frames.keys()) | set(pred_frames.keys()))
    frames = []
    for frame_id in all_frame_ids:
        frame_data = {
            'ground_truths': gt_frames.get(frame_id, []),
            'predictions': pred_frames.get(frame_id, [])
        }
        frames.append(frame_data)
    return frames

In [ ]:
import os
import glob
import numpy as np

# --- Indoor Settings ---
gt_folder = 'ground_truth/Indoor/transformed_MOT_fixed'
pred_folder = 'BoT-SORT_outputs/Indoor/transformed/merged_with_roles/'
tau = 50

# List of txt files in ground truth folder
gt_files = glob.glob(os.path.join(gt_folder, '*.txt'))

# List to store evaluation results and number of frames for each video
indoor_results = []   # Stores metrics (dictionary) for each video
indoor_frame_counts = []  # Number of frames for each video

for gt_file in gt_files:
    filename = os.path.basename(gt_file)
    pred_file = os.path.join(pred_folder, filename)
    if not os.path.exists(pred_file):
        print(f"Pred result file {pred_file} not found.Skip.")
        continue

    frames = load_video(gt_file, pred_file)
    num_frames = len(frames)
    indoor_frame_counts.append(num_frames)
    
    metrics, alphas = compute_ti_hota(frames, tau)
    indoor_results.append(metrics)
    
    print(f"Video: {filename}")
    print(f"  Number of frames: {num_frames}")
    print(f"  TI-HOTA: {metrics['THI-HOTA']:.4f}")
    print(f"  DetA: {metrics['DetA']:.4f}")
    print(f"  AssA: {metrics['AssA']:.4f}")
    print(f"  TP: {metrics['TP']:.2f}, FP: {metrics['FP']:.2f}, FN: {metrics['FN']:.2f}")
    print("-" * 50)

# Organized into a list by evaluation metrics
keys = ["TI-HOTA", "DetA", "AssA", "TP", "FP", "FN"]
metrics_data = { key: [] for key in keys }
for m in indoor_results:
    for key in keys:
        metrics_data[key].append(m[key])

print(f"===== Indoor Videos: Average ± SD (τ={tau}) =====")
if indoor_frame_counts:
    frame_avg = np.mean(indoor_frame_counts)
    frame_std = np.std(indoor_frame_counts)
    print(f"Frame Count: {frame_avg:.2f} (std: {frame_std:.2f})")
    
for key in keys:
    if metrics_data[key]:
        avg = np.mean(metrics_data[key])
        std = np.std(metrics_data[key])
        print(f"{key}: {avg:.4f} (std: {std:.4f})")
    else:
        print(f"{key}: No data")


Video: basket_S1T1_pre.txt
  Number of frames: 168
  GS-HOTA: 0.9817
  DetA: 0.9816
  AssA: 0.9817
  TP: 995.68, FP: 6.32, FN: 12.32
--------------------------------------------------
Video: basket_S1T2_pre.txt
  Number of frames: 162
  GS-HOTA: 0.7811
  DetA: 0.7705
  AssA: 0.7918
  TP: 841.89, FP: 121.11, FN: 130.11
--------------------------------------------------
Video: basket_S1T3_pre.txt
  Number of frames: 216
  GS-HOTA: 0.8271
  DetA: 0.8192
  AssA: 0.8350
  TP: 1147.74, FP: 105.26, FN: 148.26
--------------------------------------------------
Video: basket_S1T4_pre.txt
  Number of frames: 142
  GS-HOTA: 0.9857
  DetA: 0.9846
  AssA: 0.9868
  TP: 845.21, FP: 6.79, FN: 6.79
--------------------------------------------------
Video: basket_S1T5_pre.txt
  Number of frames: 236
  GS-HOTA: 0.9363
  DetA: 0.9355
  AssA: 0.9372
  TP: 1365.32, FP: 43.68, FN: 50.68
--------------------------------------------------
Video: basket_S1T6_pre.txt
  Number of frames: 142
  GS-HOTA: 0.8943
  D

In [ ]:
import os
import glob
import numpy as np

# --- Outdoor Settings ---
# Multiple ground truth folders and categories
gt_folders = [
    ('free_throw', 'ground_truth/Outdoor/MOT_files/split_transformed/free_throw'),
    ('top', 'ground_truth/Outdoor/MOT_files/split_transformed/top')
]
# Folder where the forecast results are stored (all forecast files are in this folder)
pred_folder = 'BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/with_team'
tau = 50

# List to store evaluation results and number of frames for each video
outdoor_results = []
outdoor_frame_counts = []

for folder_name, gt_folder in gt_folders:
    gt_files = glob.glob(os.path.join(gt_folder, '*.txt'))
    for gt_file in gt_files:
        filename = os.path.basename(gt_file)
        # Get a prediction result file of the same name from within pred_folder
        pred_file = os.path.join(pred_folder, filename)
        if not os.path.exists(pred_file):
            print(f"Pred result file {pred_file} not found.Skip.")
            continue

        frames = load_video(gt_file, pred_file)
        num_frames = len(frames)
        outdoor_frame_counts.append(num_frames)
        
        metrics, alphas = compute_ti_hota(frames, tau)
        outdoor_results.append(metrics)
        
        print(f"Video: {filename} (category: {folder_name})")
        print(f"  Number of frames: {num_frames}")
        print(f"  TI-HOTA: {metrics['TI-HOTA']:.4f}")
        print(f"  DetA: {metrics['DetA']:.4f}")
        print(f"  AssA: {metrics['AssA']:.4f}")
        print(f"  TP: {metrics['TP']:.2f}, FP: {metrics['FP']:.2f}, FN: {metrics['FN']:.2f}")
        print("-" * 50)

# Organized into a list by evaluation indicator
keys = ["TI-HOTA", "DetA", "AssA", "TP", "FP", "FN"]
metrics_data = { key: [] for key in keys }
for m in outdoor_results:
    for key in keys:
        metrics_data[key].append(m[key])

print(f"===== Outdoor Videos: Average ± SD (τ={tau}) =====")
if outdoor_frame_counts:
    frame_avg = np.mean(outdoor_frame_counts)
    frame_std = np.std(outdoor_frame_counts)
    print(f"Frame Count: {frame_avg:.2f} (std: {frame_std:.2f})")
    
for key in keys:
    if metrics_data[key]:
        avg = np.mean(metrics_data[key])
        std = np.std(metrics_data[key])
        print(f"{key}: {avg:.4f} (std: {std:.4f})")
    else:
        print(f"{key}: No data")

Video: IMG_0110_3.txt (category: free_throw)
  Number of frames: 882
  GS-HOTA: 0.2566
  DetA: 0.2425
  AssA: 0.2718
  TP: 2025.58, FP: 3196.42, FN: 3266.42
--------------------------------------------------
Video: IMG_0111_2.txt (category: free_throw)
  Number of frames: 512
  GS-HOTA: 0.5661
  DetA: 0.5183
  AssA: 0.6184
  TP: 2076.42, FP: 988.58, FN: 995.58
--------------------------------------------------
Video: IMG_0111_6.txt (category: free_throw)
  Number of frames: 1770
  GS-HOTA: 0.2603
  DetA: 0.2325
  AssA: 0.2915
  TP: 3943.89, FP: 6579.11, FN: 6676.11
--------------------------------------------------
Video: IMG_0111_7.txt (category: free_throw)
  Number of frames: 2312
  GS-HOTA: 0.0086
  DetA: 0.0067
  AssA: 0.0110
  TP: 180.47, FP: 13107.53, FN: 13691.53
--------------------------------------------------
Video: IMG_0112_5.txt (category: free_throw)
  Number of frames: 213
  GS-HOTA: 0.2659
  DetA: 0.2181
  AssA: 0.3242
  TP: 418.53, FP: 641.47, FN: 859.47
-------------